In [ ]:
import torch
import os
import json
import time
import random
import hashlib

%cd ..

from helpers.models.hidream.pipeline import HiDreamImagePipeline
from helpers.models.hidream.transformer import HiDreamImageTransformer2DModel
from transformers import PreTrainedTokenizerFast, LlamaForCausalLM
from lycoris import create_lycoris_from_weights
from PIL import Image

# --- Config ---
llama_repo = "/local/yada/models/Meta-Llama-3.1-8B-Instruct"
model_id = "HiDream-ai/HiDream-I1-Full"
lora_path = "/local/yada/apps/SimpleTuner-a/output/models-hidream-run04/pytorch_lora_weights.safetensors"
prompt = "a girl with silver hair, detailed background"
negative_prompt = "blurry, distorted"
width, height = 768, 1024
guidance_scale = 5.0
num_inference_steps = 28
seed = 123456
save_dir = "output/oneshot"
os.makedirs(save_dir, exist_ok=True)

# --- Load Model ---
tokenizer_4 = PreTrainedTokenizerFast.from_pretrained(llama_repo)
text_encoder_4 = LlamaForCausalLM.from_pretrained(llama_repo, output_hidden_states=True, torch_dtype=torch.bfloat16)
transformer = HiDreamImageTransformer2DModel.from_pretrained(model_id, torch_dtype=torch.bfloat16, subfolder="transformer")

pipeline = HiDreamImagePipeline.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    tokenizer_4=tokenizer_4,
    text_encoder_4=text_encoder_4,
    transformer=transformer,
)
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
pipeline.to(device)

# --- Apply LoRA ---
wrapper, _ = create_lycoris_from_weights(1.0, lora_path, pipeline.transformer)
wrapper.merge_to()

# --- Encode Prompt ---
t5_embeds, llama_embeds, negative_t5_embeds, negative_llama_embeds, pooled_embeds, negative_pooled_embeds = pipeline.encode_prompt(
    prompt=prompt, prompt_2=prompt, prompt_3=prompt, prompt_4=prompt, num_images_per_prompt=1
)

# --- Generate ---
image = pipeline(
    t5_prompt_embeds=t5_embeds,
    llama_prompt_embeds=llama_embeds,
    pooled_prompt_embeds=pooled_embeds,
    negative_t5_prompt_embeds=negative_t5_embeds,
    negative_llama_prompt_embeds=negative_llama_embeds,
    negative_pooled_prompt_embeds=negative_pooled_embeds,
    num_inference_steps=num_inference_steps,
    generator=torch.Generator(device=device).manual_seed(seed),
    width=width,
    height=height,
    guidance_scale=guidance_scale,
).images[0]


In [1]:
from procslib import get_model, print_model_keys

print_model_keys("aes")

Model Key                   | Description
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
aesthetic-siglip            | A Siglip-based aesthetic model for high-efficiency aesthetic predictions.     输入anime图片, 输出预测的siglip aesthetic score         https://github.com/discus0434/ae...
aesthetic-pixiv-compound    | Aesthetic model trained on pixiv data (of the constructed pixiv compound aesthetic score)     model at "https://bucket-public-access-uw2.s3.us-west-2.amazona...
aesthetic-clip              | Calculate and cache clip aesthetics embs and calcualtes the aesthetic scores and 0shot cls scores
aesthetic-pixai-1.1         | Aesthetic model trained on pixiv data (of the constructed pixiv compound aesthetic score)     model at "https://bucket-public-access-uw2.s3.us-west-2.amazona...
aesthetic-pixai-1.1-anatomy | Aesthetic model trained on pixiv dat